In [5]:
import stoploss as sl

# The maximum amount of PORTFOLIO_VAL to risk on a trade
MAX_RISK = 0.02

# How far to dip below support before triggering stoploss
SUPPORT_MARGIN = 0.01

# Target profit potential ratio relative to risk
TARGET_RATIO = 2

# Don't put more than this fraction of the portfolio in one stock
MAX_INVESTMENT = 0.8


In [6]:
# Value of the portfolio, used to calculate total risk
PORTFOLIO_VAL = 20000
LIQUIDITY = 20000

result = sl.shares_to_buy(23.20, 22.67)
print_result(result)

NameError: name 'PORTFOLIO_VAL' is not defined